In [1]:
import numpy as np
import json
import os
def wilson_score(pos, total, p_z=1.96):
    """
    Wilson Score
    ref：https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval
    :param pos: positive number
    :param total: total number
    :param p_z: Normal distribution quantile
    :return: wilson score
    """
    pos_rat = pos * 1. / total * 1.  # 正例比率
    score = (pos_rat + (np.square(p_z) / (2. * total))
             - ((p_z / (2. * total)) * np.sqrt(4. * total * (1. - pos_rat) * pos_rat + np.square(p_z)))) / \
            (1. + np.square(p_z) / total)
    return score

In [2]:
def Bayesian(C, M, n, s):
    return (C*M + n*s) / (n + C)

In [3]:
def calc_sentiment_score(file_name):
    file_json = []
#     print(file_name)
    with open(file_name, 'rb') as f:
            line = f.readline()
            json_data = json.loads(line)
            count = 0
            for d in json_data:
                if d['class'] == 1:
                    count += 1
            return count
        
def calc_review_score(file_name):
    file_json = []
    with open(file_name, 'rb') as f:
            line = f.readline()
#             line = [l.decode('utf8', 'ignore') for l in f.readlines()]
            json_data = json.loads(line)
            count = 0
            final_score = 0
            for d in json_data:
                count += 1
                score = (int)(d['score'].split("/")[0])
#                 print(score)
                final_score += score
            return final_score / (count * 10)
        
def save2json(dictObj, filename):
    jsonObj = json.dumps(dictObj)
    fileObject = open(filename,'w')
    fileObject.write(jsonObj)
    fileObject.close() 

In [14]:
def save2json(dictObj, filename):
    jsonObj = json.dumps(dictObj)
    fileObject = open(filename,'w')
    fileObject.write(jsonObj)
    fileObject.close()

In [15]:
prefix_raw_data = "./raw_data/"
prefix_clean_data = "./clean_data/"
g = os.walk(prefix_raw_data)
res_list = []

C = 20
M = 0.69
for path, d, filelist in g:
#     print(d)
    for filename in filelist:
        res = {}
        res['name'] = filename
#         print("filename = " + str(filename))
        file_name_raw = os.path.join(prefix_raw_data, filename)
        file_name_clean = os.path.join(prefix_clean_data, filename)
        count = calc_sentiment_score(file_name_clean)

        wilson_review_score = calc_review_score(file_name_raw)
        wilson_sentiment_score = wilson_score(count, 30, p_z=1.96)
        bayes_sentiment_score = Bayesian(C, M, 30, wilson_s)
        bayes_review_score = Bayesian(C, M, 30, wilson_review_score)
        res['wilson_review_score'] = wilson_review_score
        res['wilson_sentiment_score'] = wilson_sentiment_score
        res['bayes_sentiment_score'] = bayes_sentiment_score
        res['bayes_review_score'] = bayes_review_score
#         print(bayes_sentiment_score)
#         print(bayes_review_score)
        res['final_score'] = 10 * (bayes_sentiment_score+bayes_review_score) / 2
        res_list.append(res)

#         print(str(res['final_score']))

[]


In [16]:
save2json(res_list,"final_score.json")